Нужно создать таблицы:

ODS.Customers
 • customer_id (PK)
 • name
 • email (валидный формат)
 • phone (нормализованный: +7XXXXXXXXXX)
 • reg_date
 • country (ISO-код, например RU)

ODS.Products
 • product_id (PK)
 • product_name (default: “Unknown product” если нет названия)
 • category (default: “Other”)
 • price (default: 0.0 если NULL)

ODS.Orders
 • order_id (PK)
 • customer_id (FK → ODS.Customers)
 • product_id (FK → ODS.Products)
 • quantity (если NULL → 1)
 • order_date
 • status (ограниченный список: delivered / shipped / canceled / pending)

In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('raw_customers.csv')

In [39]:
df.head()

,customer_id,name,email,phone,reg_date,country
0,1,Kelly Sanders,ashley67@gmail.com,85084,2024-03-29,ru
1,2,Kimberly Gonzalez,anthonyhodge@yahoo.com,89034259653,2025-03-02,RUS
2,3,Charles Rush,sarah96@ramirez.com,79101763056,2024-08-24,Russia
3,4,Mariah Griffith,david53@gmail.com,62995,2024-09-19,Russia
4,5,James Grant,joneskaren@bennett.info,89795843835,2024-10-31,RUS


In [ ]:

def terminator(name):
    not_good = ['PhD', 'Miss', 'DDS']
    s = ''
    for i in name.split():
        if i not in not_good:
            s += i + ' '
    return s

df['new_name'] = df['name'].apply(terminator)
new_df = df['new_name'].str.split(' ',expand=True)
# df['last_name'] = df['new_name'].str.split()[1]

final_df = pd.concat([df,new_df],axis=1)

final_df

In [ ]:
final_df = final_df.rename(columns = {0 : 'first_name', 1:'last_name'})
final_df = final_df.drop(2, axis = 1)
final_df

In [ ]:
# def email_validator(email):
pattern = r'^(?!\.)[\w.\-\+]+@[\w\-]+\.[a-zA-Z]+'

final_df['valid_email'] = final_df["email"].str.match(pattern)

final_df

In [43]:
df.dtypes

customer_id     int64
name           object
email          object
phone           int64
reg_date       object
country        object
new_name       object
dtype: object

In [44]:
final_df["phone"] = final_df["phone"].astype(str)

In [ ]:
final_df.dtypes

In [ ]:
def phone_validator(phone):
    if len(phone) < 11:
        return None
    if phone[0] in ('7','8'):
        if phone[0] == '8':
            return '7' + phone[1:]
        return phone

final_df['valid_phone'] = final_df['phone'].apply(phone_validator)

final_df

In [ ]:
def country_naming (country):
    not_good = ['RUS', 'ru', 'RU', ]
    if country in not_good:
        country = 'Russia'
    return country

final_df['new_country'] = final_df['country'].apply(country_naming)

final_df

ODS.Products
 • product_id (PK)
 • product_name (default: “Unknown product” если нет названия)
 • category (default: “Other”)
 • price (default: 0.0 если NULL)

In [48]:
products_df = pd.read_csv('raw_products.csv')

In [49]:
products_df.head()

,product_id,product_name,category,price
0,101,NaN,Laptops,NaN
1,102,Then,Laptops,1358.09
2,103,Bit,Accessories,347.37
3,104,Cover,Laptops,85.82
4,105,Source,Accessories,1983.69


In [50]:
values = {'product_name': 'Unknown product', 'category': 'Other', 'price': 0.0}
products_df = products_df.fillna(value=values)

products_df.head()

,product_id,product_name,category,price
0,101,Unknown product,Laptops,0.00
1,102,Then,Laptops,1358.09
2,103,Bit,Accessories,347.37
3,104,Cover,Laptops,85.82
4,105,Source,Accessories,1983.69


ODS.Orders
 • order_id (PK)
 • customer_id (FK → ODS.Customers)
 • product_id (FK → ODS.Products)
 • quantity (если NULL → 1)
 • order_date
 • status (ограниченный список: delivered / shipped / canceled / pending)S

In [51]:
orders_df = pd.read_csv('raw_orders.csv')
orders_df.head()


,order_id,customer_id,product_id,quantity,order_date,status
0,5001,9,122.0,2.0,2025-04-13,delivered
1,5002,11,129.0,NaN,2025-03-02,shipped
2,5003,7,130.0,NaN,2025-07-18,delivered
3,5004,3,126.0,1.0,2024-09-15,delivered
4,5005,16,114.0,2.0,2025-02-05,shipped


In [ ]:
values = {'quantity': 1}
orders_df = orders_df.fillna(value=values)

orders_df

In [ ]:
def pendingnator(status):
    status_good = ['delivered', 'shipped', 'canceled']
    if status not in status_good:
        status = 'pending'
    return status

orders_df['new_status'] = orders_df['status'].apply(pendingnator)

orders_df

In [54]:
# orders_df.join()
order_prod_tab = pd.merge(orders_df, products_df, left_on='product_id', right_on='product_id', how='left')


In [55]:
mega_tab = pd.merge(order_prod_tab, final_df, left_on='customer_id', right_on='customer_id', how='left').drop(['status', 'phone', 'country', 'name'],  axis=1)
mega_tab.head()


,order_id,customer_id,product_id,quantity,order_date,new_status,product_name,category,price,email,reg_date,new_name,first_name,last_name,valid_email,valid_phone,new_country
0,5001,9,122.0,2.0,2025-04-13,delivered,Right,Other,0.00,thomas24@hotmail.com,2025-05-26,Jennifer Simmons,Jennifer,Simmons,True,79502514879,Russia
1,5002,11,129.0,1.0,2025-03-02,shipped,Too,Smartphones,643.98,llawson@@mail.ru,2023-09-30,Ryan Wolfe,Ryan,Wolfe,False,79040542247,Russia
2,5003,7,130.0,1.0,2025-07-18,delivered,Unknown product,Accessories,0.00,fieldscynthia@hotmail.com,2024-08-05,Michael Harris,Michael,Harris,True,79258309953,Russia
3,5004,3,126.0,1.0,2024-09-15,delivered,Job,Other,1047.51,sarah96@ramirez.com,2024-08-24,Charles Rush,Charles,Rush,True,79101763056,Russia
4,5005,16,114.0,2.0,2025-02-05,shipped,Unknown product,Other,1024.22,tracymyers@hester.net,2025-01-10,Matthew Ward,Matthew,Ward,True,79576154882,Russia


In [56]:
null_data = mega_tab[mega_tab.isnull().any(axis=1)]
null_data


,order_id,customer_id,product_id,quantity,order_date,new_status,product_name,category,price,email,reg_date,new_name,first_name,last_name,valid_email,valid_phone,new_country
5,5006,23,101.0,2.0,2025-06-21,canceled,Unknown product,Laptops,0.00,merceryolanda@gomez-anderson.com,2025-03-12,Jeffrey Martinez,Jeffrey,Martinez,True,None,Russia
6,5007,18,118.0,1.0,2025-02-10,pending,Unknown product,Laptops,1431.54,jedwards@gmail.com,2024-08-29,Tracy Fisher,Tracy,Fisher,True,None,Russia
9,5010,6,110.0,1.0,2025-03-21,pending,Can,Smartphones,1899.22,ulewis@pacheco.net,2023-11-12,Gregory Baird,Gregory,Baird,True,None,Russia
16,5017,1,117.0,1.0,2024-10-22,pending,Unknown product,Laptops,713.72,ashley67@gmail.com,2024-03-29,Kelly Sanders,Kelly,Sanders,True,None,Russia
17,5018,1,124.0,3.0,2024-11-22,delivered,Unknown product,Smartphones,97.55,ashley67@gmail.com,2024-03-29,Kelly Sanders,Kelly,Sanders,True,None,Russia
19,5020,10,NaN,2.0,2025-08-06,canceled,NaN,NaN,NaN,monica07@hotmail.com,2025-01-07,Todd Moore,Todd,Moore,True,79293750665,Russia
25,5026,4,120.0,1.0,2024-11-26,canceled,Stay,Other,1504.16,david53@gmail.com,2024-09-19,Mariah Griffith,Mariah,Griffith,True,None,Russia
27,5028,14,NaN,2.0,2024-09-18,shipped,NaN,NaN,NaN,ggoodwin@yahoo.com,2024-03-16,Ray Hansen,Ray,Hansen,True,79509258538,Russia
31,5032,1,101.0,1.0,2025-03-16,shipped,Unknown product,Laptops,0.00,ashley67@gmail.com,2024-03-29,Kelly Sanders,Kelly,Sanders,True,None,Russia
32,5033,6,125.0,3.0,2025-02-01,canceled,Clear,Accessories,1245.62,ulewis@pacheco.net,2023-11-12,Gregory Baird,Gregory,Baird,True,None,Russia


In [57]:
# mega_tab.drop(['5020', '5028', '5042'],  axis=0)

mega_tab = mega_tab.loc[mega_tab['product_name'].notna()]
null_data = mega_tab[mega_tab.isnull().any(axis=1)]
null_data.head()

,order_id,customer_id,product_id,quantity,order_date,new_status,product_name,category,price,email,reg_date,new_name,first_name,last_name,valid_email,valid_phone,new_country
5,5006,23,101.0,2.0,2025-06-21,canceled,Unknown product,Laptops,0.00,merceryolanda@gomez-anderson.com,2025-03-12,Jeffrey Martinez,Jeffrey,Martinez,True,None,Russia
6,5007,18,118.0,1.0,2025-02-10,pending,Unknown product,Laptops,1431.54,jedwards@gmail.com,2024-08-29,Tracy Fisher,Tracy,Fisher,True,None,Russia
9,5010,6,110.0,1.0,2025-03-21,pending,Can,Smartphones,1899.22,ulewis@pacheco.net,2023-11-12,Gregory Baird,Gregory,Baird,True,None,Russia
16,5017,1,117.0,1.0,2024-10-22,pending,Unknown product,Laptops,713.72,ashley67@gmail.com,2024-03-29,Kelly Sanders,Kelly,Sanders,True,None,Russia
17,5018,1,124.0,3.0,2024-11-22,delivered,Unknown product,Smartphones,97.55,ashley67@gmail.com,2024-03-29,Kelly Sanders,Kelly,Sanders,True,None,Russia



После загрузки данных в ODS нужно построить SQL-витрины:
 1. MART.CustomerSales — сумма и количество заказов на каждого клиента.
 2. MART.ProductSales — продажи по каждому товару.
 3. MART.CountrySales — продажи по странам.

 1. Найди топ-3 клиента по выручке.
 2. Посчитай, сколько заказов было отменено.
 3. Найди категорию товаров с максимальной выручкой.
 4. Составь таблицу retention: сколько клиентов вернулись за покупкой повторно.

In [58]:
mega_tab.to_csv('orders_table.csv')

In [ ]:
mega_tab


                ///ВТОРАЯ ЧАСТЬ///

In [60]:
data_frame_customers = pd.read_csv('raw_generated_data_archive/raw_customers_2.csv')
data_frame_orders = pd.read_csv('raw_generated_data_archive/raw_orders_2.csv')
data_frame_products = pd.read_csv('raw_generated_data_archive/raw_products_2.csv')

In [ ]:
data_frame_customers
# data_frame_orders.head()
# data_frame_products.head()

In [ ]:
def terminator(name):
    not_good = ['PhD', 'Miss', 'DDS', 'Mrs.', 'II']
    s = ''
    for i in name.split():
        if i not in not_good:
            s += i + ' '
    return s

data_frame_customers['new_name'] = data_frame_customers['name'].apply(terminator)
new_df = data_frame_customers['new_name'].str.split(' ',expand=True)


data_frame = pd.concat([data_frame_customers, new_df],axis=1)
data_frame = data_frame.rename(columns = {0 : 'first_name', 1:'last_name'})
data_frame = data_frame.drop(2, axis = 1)
data_frame



In [ ]:
pattern = r'^(?!\.)[\w.\-\+]+@[\w\-]+\.[a-zA-Z]+'

data_frame['valid_email'] = data_frame["email"].str.match(pattern)

data_frame

In [ ]:
def country_naming (country):
    dict_countries = {
        'Russia': ['RUS', 'ru', 'RU', 'Russia'],
        'USA': ['U.S.', 'USA', 'United States'],
        'Brasil': ['BR', 'Brasil'],
        'India': ['IN', 'India'],
        'China': ['CN', 'China'],
        'Germany': ['Germany'],
        'France': ['FR'],
    }
    for key, value in dict_countries.items():
        if country in value:
            country = key
            break
    return country

data_frame['new_country'] = data_frame['country'].apply(country_naming)

data_frame



In [65]:
data_frame_orders = data_frame_orders.loc[data_frame_orders['order_date'].notna()]
data_frame_orders

,order_id,customer_id,product_id,order_date,status
0,1,43,15,2025-08-03,delivered
1,2,11,17,2025-06-25,in progress
2,3,33,18,2025-06-15,in progress
3,4,7,18,2025-05-30,shipped
5,6,16,7,2025-07-24,canceled
...,...,...,...,...,...
74,75,19,11,2025-06-04,canceled
76,77,8,8,2025-06-29,shipped
77,78,36,1,2025-07-05,in progress
78,79,42,18,2025-08-07,pending


In [101]:
def status_naming (status):
    not_dlvr = ['DELIVRD']
    if status in not_dlvr:
        status = 'delivered'
    return status

data_frame_orders['new_status'] = data_frame_orders['status'].apply(status_naming)

data_frame_orders

,order_id,customer_id,product_id,order_date,status,quantity,new_status
0,1,43,15,2025-08-03,delivered,1,delivered
1,2,11,17,2025-06-25,in progress,1,in progress
2,3,33,18,2025-06-15,in progress,1,in progress
3,4,7,18,2025-05-30,shipped,1,shipped
5,6,16,7,2025-07-24,canceled,1,canceled
...,...,...,...,...,...,...,...
74,75,19,11,2025-06-04,canceled,1,canceled
76,77,8,8,2025-06-29,shipped,1,shipped
77,78,36,1,2025-07-05,in progress,1,in progress
78,79,42,18,2025-08-07,pending,1,pending


In [ ]:
import random 
def random_phone():
    phone = '79'
    for i in range(9):
        phone += str(random.randint(0, 9))
    return phone

# data_frame['phone'] = random_phone()
# data_frame

data_frame['valid_phone']=[random_phone() for _ in range(len(data_frame))]
data_frame

In [90]:
def random_data():
    day = random.randint(1, 30)
    if day <= 9:
        day = f'0{day}'
    date = f'2025-0{random.randint(1, 9)}-{day}'
    return date

In [ ]:
data_frame['reg_date']=[random_data() for _ in range(len(data_frame))]
data_frame

In [72]:
data_frame.columns

Index(['customer_id', 'name', 'email', 'country', 'new_name', 'first_name',
       'last_name', 'valid_email', 'new_country', 'vaide_phone', 'reg_date'],
      dtype='object')

In [78]:
final_df.columns

Index(['customer_id', 'name', 'email', 'reg_date', 'country', 'new_name',
       'first_name', 'last_name', 'valid_email', 'valid_phone', 'new_country'],
      dtype='object')

In [85]:
# final_df.drop('phone', axis=0)
# del final_df['phone']
del data_frame['vaide_phone']

In [86]:
big_customers_data = pd.concat([data_frame, final_df], axis=0)
big_customers_data

,customer_id,name,email,country,new_name,first_name,last_name,valid_email,new_country,reg_date,valid_phone
0,1,Brandon Russell,grayanna@christian-moore.com,RU,Brandon Russell,Brandon,Russell,True,Russia,2025-03-30,79584854643
1,2,Mia Martin,johnsnicholas@gmail.com,IN,Mia Martin,Mia,Martin,True,India,2025-07-16,79019923782
2,3,Jerome Whitehead,davidknox@pruitt-berry.com,RU,Jerome Whitehead,Jerome,Whitehead,True,Russia,2025-03-13,79297100535
3,4,Andrew Torres,lorifowler@hotmail.com,Brasil,Andrew Torres,Andrew,Torres,True,Brasil,2025-01-13,79267080354
4,5,Emily Sanders,ameyer@yahoo.com,RU,Emily Sanders,Emily,Sanders,True,Russia,2025-09-20,79263963526
...,...,...,...,...,...,...,...,...,...,...,...
25,26,Tara Miller,ososa@nelson-garcia.com,ru,Tara Miller,Tara,Miller,True,Russia,2025-06-28,79017671811
26,27,Thomas Saunders,richard22@yahoo.com,ru,Thomas Saunders,Thomas,Saunders,True,Russia,2025-06-24,None
27,28,Daniel Ayala,dawn14@gmail.com,Russia,Daniel Ayala,Daniel,Ayala,True,Russia,2024-12-13,79838177711
28,29,Nicholas Watson,nrichards@hotmail.com,Russia,Nicholas Watson,Nicholas,Watson,True,Russia,2025-04-26,79436249832


In [88]:
big_customers_data['customer_id'] = [i for i in range(1, len(big_customers_data)+1)]

In [89]:
big_customers_data

,customer_id,name,email,country,new_name,first_name,last_name,valid_email,new_country,reg_date,valid_phone
0,1,Brandon Russell,grayanna@christian-moore.com,RU,Brandon Russell,Brandon,Russell,True,Russia,2025-03-30,79584854643
1,2,Mia Martin,johnsnicholas@gmail.com,IN,Mia Martin,Mia,Martin,True,India,2025-07-16,79019923782
2,3,Jerome Whitehead,davidknox@pruitt-berry.com,RU,Jerome Whitehead,Jerome,Whitehead,True,Russia,2025-03-13,79297100535
3,4,Andrew Torres,lorifowler@hotmail.com,Brasil,Andrew Torres,Andrew,Torres,True,Brasil,2025-01-13,79267080354
4,5,Emily Sanders,ameyer@yahoo.com,RU,Emily Sanders,Emily,Sanders,True,Russia,2025-09-20,79263963526
...,...,...,...,...,...,...,...,...,...,...,...
25,66,Tara Miller,ososa@nelson-garcia.com,ru,Tara Miller,Tara,Miller,True,Russia,2025-06-28,79017671811
26,67,Thomas Saunders,richard22@yahoo.com,ru,Thomas Saunders,Thomas,Saunders,True,Russia,2025-06-24,None
27,68,Daniel Ayala,dawn14@gmail.com,Russia,Daniel Ayala,Daniel,Ayala,True,Russia,2024-12-13,79838177711
28,69,Nicholas Watson,nrichards@hotmail.com,Russia,Nicholas Watson,Nicholas,Watson,True,Russia,2025-04-26,79436249832


In [94]:
data_frame_orders['quantity'] = 1
data_frame_orders

,order_id,customer_id,product_id,order_date,status,quantity
0,1,43,15,2025-08-03,delivered,1
1,2,11,17,2025-06-25,in progress,1
2,3,33,18,2025-06-15,in progress,1
3,4,7,18,2025-05-30,shipped,1
5,6,16,7,2025-07-24,canceled,1
...,...,...,...,...,...,...
74,75,19,11,2025-06-04,canceled,1
76,77,8,8,2025-06-29,shipped,1
77,78,36,1,2025-07-05,in progress,1
78,79,42,18,2025-08-07,pending,1


In [103]:

orders_df.columns

Index(['order_id', 'customer_id', 'product_id', 'quantity', 'order_date',
       'status', 'new_status'],
      dtype='object')

In [102]:
data_frame_orders.columns

Index(['order_id', 'customer_id', 'product_id', 'order_date', 'status',
       'quantity', 'new_status'],
      dtype='object')

In [ ]:
big_orders_data = pd.concat([data_frame_orders, orders_df], axis=0)
big_orders_data

In [ ]:
big_products_data = pd.concat([data_frame_products, products_df], axis=0)
big_products_data

In [ ]:
order_prod_tab_2 = pd.merge(big_orders_data, big_products_data, left_on='product_id', right_on='product_id', how='left')
big_tab = pd.merge(order_prod_tab_2, big_customers_data, left_on='customer_id', right_on='customer_id', how='left').drop(['status', 'name', 'country'],  axis=1)
big_tab.head()

In [ ]:
def not_valid_email_terminator(n, y):
    # print (n)
    if y == True:
        return n
big_tab['e_mail'] = big_tab.apply(lambda x: not_valid_email_terminator(x.email, x.valid_email), axis=1)



,order_id,customer_id,product_id,order_date,quantity,new_status,product_name,category,price,email,new_name,first_name,last_name,valid_email,new_country,reg_date,valid_phone,e_mail
0,1,43,15.0,2025-08-03,1.0,delivered,Design,Smrtphone,1828.31,sarah96@ramirez.com,Charles Rush,Charles,Rush,True,Russia,2024-08-24,79101763056,sarah96@ramirez.com
1,2,11,17.0,2025-06-25,1.0,in progress,Edge,Laptops,682.07,ymerritt@bennett.com,Katrina Mann,Katrina,Mann,True,USA,2025-07-12,79009549974,ymerritt@bennett.com
2,3,33,18.0,2025-06-15,1.0,in progress,Easy,Smartphones,1606.53,jessicakelly,Jessica Kelly,Jessica,Kelly,False,Brasil,2025-03-19,79095975148,None
3,4,7,18.0,2025-05-30,1.0,shipped,Easy,Smartphones,1606.53,brendadecker@smith-bell.com,Jasmine Osborne,Jasmine,Osborne,True,Germany,2025-08-11,79192420456,brendadecker@smith-bell.com
4,6,16,7.0,2025-07-24,1.0,canceled,Believe,Other,2067.33,gardnermiguel@hotmail.com,Jose Hawkins,Jose,Hawkins,True,France,2025-02-18,79616991988,gardnermiguel@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,5046,19,107.0,2025-05-04,1.0,canceled,Environmental,Other,729.60,zvasquez@scott.info,Brandi Zamora,Brandi,Zamora,True,China,2025-08-11,79304390326,zvasquez@scott.info
114,5047,6,109.0,2024-11-07,2.0,shipped,Unknown product,Other,1713.74,kbarrett@rogers-myers.com,Sarah Brewer,Sarah,Brewer,True,Brasil,2025-06-19,79321488243,kbarrett@rogers-myers.com
115,5048,6,122.0,2025-04-16,3.0,delivered,Right,Other,0.00,kbarrett@rogers-myers.com,Sarah Brewer,Sarah,Brewer,True,Brasil,2025-06-19,79321488243,kbarrett@rogers-myers.com
116,5049,29,117.0,2025-06-19,2.0,shipped,Unknown product,Laptops,713.72,justin65@williams-smith.com,James Watts,James,Watts,True,Russia,2025-09-19,79895990279,justin65@williams-smith.com


In [122]:
big_tab.drop(['email', 'valid_email'], axis=1)

,order_id,customer_id,product_id,order_date,quantity,new_status,product_name,category,price,new_name,first_name,last_name,new_country,reg_date,valid_phone,e_mail
0,1,43,15.0,2025-08-03,1.0,delivered,Design,Smrtphone,1828.31,Charles Rush,Charles,Rush,Russia,2024-08-24,79101763056,sarah96@ramirez.com
1,2,11,17.0,2025-06-25,1.0,in progress,Edge,Laptops,682.07,Katrina Mann,Katrina,Mann,USA,2025-07-12,79009549974,ymerritt@bennett.com
2,3,33,18.0,2025-06-15,1.0,in progress,Easy,Smartphones,1606.53,Jessica Kelly,Jessica,Kelly,Brasil,2025-03-19,79095975148,None
3,4,7,18.0,2025-05-30,1.0,shipped,Easy,Smartphones,1606.53,Jasmine Osborne,Jasmine,Osborne,Germany,2025-08-11,79192420456,brendadecker@smith-bell.com
4,6,16,7.0,2025-07-24,1.0,canceled,Believe,Other,2067.33,Jose Hawkins,Jose,Hawkins,France,2025-02-18,79616991988,gardnermiguel@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,5046,19,107.0,2025-05-04,1.0,canceled,Environmental,Other,729.60,Brandi Zamora,Brandi,Zamora,China,2025-08-11,79304390326,zvasquez@scott.info
114,5047,6,109.0,2024-11-07,2.0,shipped,Unknown product,Other,1713.74,Sarah Brewer,Sarah,Brewer,Brasil,2025-06-19,79321488243,kbarrett@rogers-myers.com
115,5048,6,122.0,2025-04-16,3.0,delivered,Right,Other,0.00,Sarah Brewer,Sarah,Brewer,Brasil,2025-06-19,79321488243,kbarrett@rogers-myers.com
116,5049,29,117.0,2025-06-19,2.0,shipped,Unknown product,Laptops,713.72,James Watts,James,Watts,Russia,2025-09-19,79895990279,justin65@williams-smith.com


In [124]:
big_tab.to_csv('orders_table_2.csv', index=False)

In [1]:
big_tab.head()


NameError: name 'big_tab' is not defined